# Mnist classification pipeline using Sagemaker

The `mnist-classification-pipeline.py` sample runs a pipeline to train a classficiation model using Kmeans with MNIST dataset on Sagemaker.

We will have all required steps here and for other details like how to get source data, please check [documentation](https://github.com/kubeflow/pipelines/tree/master/samples/contrib/aws-samples/mnist-kmeans-sagemaker).


This sample is based on the [Train a Model with a Built-in Algorithm and Deploy it](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1.html).

The sample trains and deploy a model based on the [MNIST dataset](http://www.deeplearning.net/tutorial/gettingstarted.html).



## Prerequisite

Copy `data` and `valid_data.csv` into your S3 bucket.

In [1]:
!pip install boto3

You are using pip version 19.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import boto3
AWS_ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
AWS_REGION = 'us-west-2'

S3_BUCKET = 'sagemaker-{}-{}'.format(AWS_REGION, AWS_ACCOUNT_ID)
print(S3_BUCKET)

sagemaker-us-west-2-173153674984


In [3]:
!aws s3 cp s3://kubeflow-pipeline-data/mnist_kmeans_example/data s3://$S3_BUCKET/mnist_kmeans_example/data

!aws s3 cp s3://kubeflow-pipeline-data/mnist_kmeans_example/input/valid_data.csv s3://$S3_BUCKET/mnist_kmeans_example/input/

copy: s3://kubeflow-pipeline-data/mnist_kmeans_example/data to s3://sagemaker-us-west-2-173153674984/mnist_kmeans_example/data
copy: s3://kubeflow-pipeline-data/mnist_kmeans_example/input/valid_data.csv to s3://sagemaker-us-west-2-173153674984/mnist_kmeans_example/input/valid_data.csv


Install Kubeflow Pipelines SDK

In [4]:
!pip install https://storage.googleapis.com/ml-pipeline/release/0.1.29/kfp.tar.gz --upgrade

    100% |████████████████████████████████| 92kB 2.5MB/s ta 0:00:011
    100% |████████████████████████████████| 1.4MB 25.9MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 33.0MB/s ta 0:00:01
  Stored in directory: /tmp/pip-ephem-wheel-cache-5vwjukbk/wheels/81/b7/33/00ef9dd992b13add014c4875a2c130d9d70288127a793c4af6


  Stored in directory: /root/.cache/pip/wheels/07/13/f3/31e9e1a25e10b8c3d04df74a01f4dbcf16a4119272cd41ba7a
  Stored in directory: /root/.cache/pip/wheels/bd/5b/6b/20cdc06ddb10caa3a86f5804eb9a90122ae8de0bcf19a468d8
  Stored in directory: /root/.cache/pip/wheels/2b/67/89/414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
  Stored in directory: /root/.cache/pip/wheels/54/f9/95/099544e9f879f719b14cf567fabb5aa7984263df0f025f3eef
Successfully built kfp kfp-server-api argo-models tabulate wrapt
kubeflow-tfjob 0.1.3 has requirement kubernetes>=10.0.1, but you'll have kubernetes 9.0.0 which is incompatible.
kubeflow-pytorchjob 0.1.3 has requirement kubernetes>=10.0.1, but you'll have kubernetes 9.0.0 which is incompatible.
kubeflow-fairing 0.7.1 has requirement kubernetes>=10.0.1, but you'll have kubernetes 9.0.0 which is incompatible.
kfserving 0.2.2.1 has requirement kubernetes>=10.0.1, but you'll have kubernetes 9.0.0 which is incompatible.
  Found existing installation: kubernetes 10.0.1
  

## Build pipeline

1. Run the following command to load Kubeflow Pipelines SDK

In [5]:
import kfp
from kfp import components
from kfp import dsl
from kfp.aws import use_aws_secret

2. Load reusable sagemaker components.

In [6]:
sagemaker_train_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/train/component.yaml')
sagemaker_model_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/model/component.yaml')
sagemaker_deploy_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/deploy/component.yaml')
sagemaker_batch_transform_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/batch_transform/component.yaml')

3. Create pipeline. 

We will create a training job first. Once training job is done, it will persist trained model to S3. 

Then a job will be kicked off to create a `Model` manifest in Sagemaker. 

With this model, batch transformation job can use it to predict on other datasets, prediction service can create an endpoint using it.


> Note: remember to use your **role_arn** to successfully run the job.

> Note: If you use a different region, please replace `us-west-2` with your region. 

> Note: ECR Images for k-means algorithm

|Region| ECR Image|
|------|----------|
|us-west-1|632365934929.dkr.ecr.us-west-1.amazonaws.com|
|us-west-2|174872318107.dkr.ecr.us-west-2.amazonaws.com|
|us-east-1|382416733822.dkr.ecr.us-east-1.amazonaws.com|
|us-east-2|404615174143.dkr.ecr.us-east-2.amazonaws.com|
|us-gov-west-1|226302683700.dkr.ecr.us-gov-west-1.amazonaws.com|
|ap-east-1|286214385809.dkr.ecr.ap-east-1.amazonaws.com|
|ap-northeast-1|351501993468.dkr.ecr.ap-northeast-1.amazonaws.com|
|ap-northeast-2|835164637446.dkr.ecr.ap-northeast-2.amazonaws.com|
|ap-south-1|991648021394.dkr.ecr.ap-south-1.amazonaws.com|
|ap-southeast-1|475088953585.dkr.ecr.ap-southeast-1.amazonaws.com|
|ap-southeast-2|712309505854.dkr.ecr.ap-southeast-2.amazonaws.com|
|ca-central-1|469771592824.dkr.ecr.ca-central-1.amazonaws.com|
|eu-central-1|664544806723.dkr.ecr.eu-central-1.amazonaws.com|
|eu-north-1|669576153137.dkr.ecr.eu-north-1.amazonaws.com|
|eu-west-1|438346466558.dkr.ecr.eu-west-1.amazonaws.com|
|eu-west-2|644912444149.dkr.ecr.eu-west-2.amazonaws.com|
|eu-west-3|749696950732.dkr.ecr.eu-west-3.amazonaws.com|
|me-south-1|249704162688.dkr.ecr.me-south-1.amazonaws.com|
|sa-east-1|855470959533.dkr.ecr.sa-east-1.amazonaws.com|

## Note: remember to use your `SAGEMAKER_ROLE_ARN` to successfully run the job.

In [7]:
# Configure your s3 bucket.
S3_PIPELINE_PATH='s3://{}/mnist_kmeans_example'.format(S3_BUCKET)

# Configure your Sagemaker execution role from the SageMaker Console Notebook Instance Tab
SAGEMAKER_ROLE_ARN='arn:aws:iam::173153674984:role/TeamRole'

@dsl.pipeline(
    name='MNIST Classification pipeline',
    description='MNIST Classification using KMEANS in SageMaker'
)
def mnist_classification(region='us-west-2',
    image='174872318107.dkr.ecr.us-west-2.amazonaws.com/kmeans:1',
    dataset_path=S3_PIPELINE_PATH + '/data',
    instance_type='ml.c4.8xlarge',
    instance_count='2',
    volume_size='50',
    model_output_path=S3_PIPELINE_PATH + '/model',
    batch_transform_input=S3_PIPELINE_PATH + '/input',
    batch_transform_ouput=S3_PIPELINE_PATH + '/output',
    role_arn=SAGEMAKER_ROLE_ARN
    ):

    training = sagemaker_train_op(
        region=region,
        image=image,
        instance_type=instance_type,
        instance_count=instance_count,
        volume_size=volume_size,
        dataset_path=dataset_path,
        model_artifact_path=model_output_path,
        role=role_arn,
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

    create_model = sagemaker_model_op(
        region=region,
        image=image,
        model_artifact_url=training.outputs['model_artifact_url'],
        model_name=training.outputs['job_name'],
        role=role_arn
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

    prediction = sagemaker_deploy_op(
        region=region,
        model_name=create_model.output
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

    batch_transform = sagemaker_batch_transform_op(
        region=region,
        model_name=create_model.output,
        input_location=batch_transform_input,
        output_location=batch_transform_ouput
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

4. Compile your pipeline

In [8]:
kfp.compiler.Compiler().compile(mnist_classification, 'mnist-classification-pipeline.zip')

5. Deploy your pipeline

In [9]:
client = kfp.Client()
aws_experiment = client.create_experiment(name='aws')
my_run = client.run_pipeline(aws_experiment.id, 'mnist-classification-pipeline', 
  'mnist-classification-pipeline.zip')

## Training

_Note:  The above training job may take 5-10 minutes.  Please be patient._

In the meantime, open the SageMaker Console to monitor the progress of your training job.

![](/images/kubeflow-sagemaker-training-job.png)

## Prediction
Open the SageMaker console and find your endpoint name. Please check dataset section to get train_set.

Once your pipeline is done, you can find sagemaker endpoint name and replace ENDPOINT_NAME value with your newly created endpoint name.



## Find your Endpoint name in AWS Console

Open AWS console and enter Sagemaker service, find the endpoint name as the following picture shows.

![download-pipeline](images/sm-endpoint.jpg)

In [11]:
import pickle, gzip, numpy, urllib.request, json
from urllib.parse import urlparse
import json
import io
import boto3

# Replace the endpoint name with yours.
ENDPOINT_NAME='Endpoint-20200222203140-PLYF'

# Load the dataset
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

# Simple function to create a csv from our numpy array
def np2csv(arr):
    csv = io.BytesIO()
    numpy.savetxt(csv, arr, delimiter=',', fmt='%g')
    return csv.getvalue().decode().rstrip()

runtime = boto3.Session(region_name='us-west-2').client('sagemaker-runtime')

payload = np2csv(train_set[0][30:31])

response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                   ContentType='text/csv',
                                   Body=payload)
result = json.loads(response['Body'].read().decode())
print(result)

{'predictions': [{'distance_to_cluster': 7.251535892486572, 'closest_cluster': 5.0}]}


## Clean up

Go to Sagemaker console and delete `endpoint` and `model`.